In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
from importlib import reload

In [4]:
import drivers, lib
from time import sleep
import pickle

In [5]:
from lib import data_management as dm

In [6]:
# reload(drivers.Agilent_EXA)
from drivers.Agilent_EXA import *
from drivers.Agilent_PNA_L import *
# reload(drivers.KeysightAWG)
from drivers.KeysightAWG import KeysightAWG, WaveformType
from drivers.E8257D import MXG, EXG

In [7]:
exa = Agilent_EXA_N9010A("EXA")
ro_awg = KeysightAWG("AWG3")
q_awg = KeysightAWG("AWG2")
#ro_lo = EXG("EXG")

In [11]:
q_lo = MXG("MXG")

In [61]:
ro_lo = Agilent_PNA_L("PNA-L2")
ro_lo.set_frequency = ro_lo.set_center
ro_lo.set_output_state = lambda x: x
ro_lo.set_span(0)
ro_lo.set_nop(1)
ro_lo.sweep_continuous()

In [48]:
reload(lib.iq_mixer_calibration)
from lib.iq_mixer_calibration import *

##  Readout resonator

In [16]:
ro_resonator_frequency = 7.047e9
if_offset = 0

In [62]:
cal = IQCalibrator(ro_awg, exa, ro_lo, "TEST", -6)
res = cal.calibrate(lo_frequency=ro_resonator_frequency, if_frequency=if_offset, lo_power=0,
                    ssb_power=-30, waveform_resolution=1, iterations=3, minimize_iterlimit=20,sa_res_bandwidth=500,)
#                     initial_guess=res.get_optimization_results()[0])
print(res)

Calibration data for mixer TEST
Mixer parameters: {'mixer_id': 'TEST', 'iq_attenuation': -6}
Radiation parameters: {'lo_frequency': 7047000000.0, 'lo_power': 0, 'if_frequency': 0, 'ssb_power': -30, 'waveform_resolution': 1}
Optimization results: {'dc': -96.343833923339844, 'dc_open': [-29.989290237426758]}
Optimization parameters {'dc_offsets': array([ 0.01168432, -0.06140434]), 'dc_offsets_open': array([ 0.34890039,  0.34914089]), 'if_offsets': None, 'if_amplitudes': None, 'if_phase': None}
Optimization time: 0 h 0 m 20.52 s
Finished at: 2017-05-10 21:34:20.236870


In [63]:
dm.save_IQMX_calibration(res)

In [7]:
calibrations=dm.load_IQMX_calibration_database("TEST", -20)
res = calibrations.get(frozenset(dict(lo_power=10, ssb_power=-30, lo_frequency=7e9, 
                                      if_frequency=10e6, waveform_resolution=0.1).items()))

## Qubit

In [20]:
qubit_frequency = 7.9e9
if_frequency = 50e6

In [43]:
cal = IQCalibrator(q_awg, exa, q_lo, "TESTQ", -6)
res = cal.calibrate(lo_frequency=qubit_frequency+if_frequency, if_frequency=if_frequency, lo_power=15,
                    ssb_power=-10, waveform_resolution=.1, iterations=3, minimize_iterlimit=20,sa_res_bandwidth=500,)
#                     initial_guess=res.get_optimization_results()[0])}
print(res)

Phase:  1.4779 [ -11.723 -68.401 -77.051 ]             ] loss: -59.476388931274414          Calibration data for mixer TESTQ
Mixer parameters: {'mixer_id': 'TESTQ', 'iq_attenuation': -6}
Radiation parameters: {'lo_frequency': 7950000000.0, 'lo_power': 15, 'if_frequency': 50000000.0, 'ssb_power': -10, 'waveform_resolution': 0.1}
Optimization results: {'dc': -80.430526733398438, 'if': [-11.723274230957031, -68.40145874023438, -77.05072784423828]}
Optimization parameters {'dc_offsets': array([ 0.04308032, -0.08657295]), 'dc_offsets_open': None, 'if_offsets': array([ 0.05021537, -0.09219981]), 'if_amplitudes': array([ 0.73356198,  0.72048027]), 'if_phase': array([ 4.64386468])}
Optimization time: 0 h 1 m 13.31 s
Finished at: 2017-05-10 20:32:57.958256


In [44]:
dm.save_IQMX_calibration(res)

In [53]:
q_lo.visa_instr.close()